# import packages

In [5]:
library(DESeq2)
library(data.table)
library(dplyr)
library(ChIPseeker)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(EnsDb.Hsapiens.v75)
library(clusterProfiler)
library(AnnotationDbi)
library(XML)
library(org.Hs.eg.db)
library(ggplot2)
library(gridExtra)
library(IRdisplay)
library(pbapply)
library(ChIPpeakAnno)
library(sva)
library(TCGAbiolinks) 
library(IRdisplay)
library(ggpubr)
library(DiffBind)
library(Seurat)
library(kableExtra)
library(tidyverse)
library(clusterProfiler)
library(enrichplot)
library(readxl)
library(umap)
library(corrplot)
library(ggrepel)
library(msigdbr)
library(ggraph)
library(tidygraph)
library(GEOquery)
library(fgsea)
library(ggvenn)
library(ComplexHeatmap)
library(ggVennDiagram)
library(venneuler)
library(sva)
library(DT)
library(tools)
library(tidyverse)
library(maSigPro)
library(RSQLite)

library(edgeR)

#txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

options(width = 188)

base_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/TE/enhancer/'
source('~/script/utils.r')
setwd(base_dir)

path = '/usr/NX/bin:/cm/local/apps/environment-modules/4.5.3//bin:/hpcf/lsf/lsf_prod/10.1/linux3.10-glibc2.17-x86_64/etc:/hpcf/lsf/lsf_prod/10.1/linux3.10-glibc2.17-x86_64/bin:/usr/lpp/mmfs/bin:/usr/lpp/mmfs/lib:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/sbin:/usr/sbin:/cm/local/apps/environment-modules/4.5.3/bin:/usr/sbin:/sbin:/bin:/usr/bin:/etc:/usr/ucb:/usr/local/bin:/usr/local/local_dfs/bin:/usr/bin/X11:/usr/local/sas:/home/whu78/edirect:/home/whu78/.local/bin:/home/whu78/bin'
path = paste0(Sys.getenv('PATH'), ':', path)
Sys.setenv(PATH = path)


# import design table

In [7]:
dsn = fread('dsn_sample_atac_may28.tsv')

In [9]:
colnames(dsn)

[1] "base_sra_file"      "sample_name"        "cell_type"          "antibody"           "srr_id"             "sample_dir"         "sra_file"           "fname"             
 [9] "full_fname"         "ori_dir"            "cmd"                "prefetch_cmd"       "fastqdump_cmd"      "dump_r"             "dump_r1"            "dump_r2"           
[17] "gz_r_cmd"           "gz_r1_cmd"          "gz_r2_cmd"          "r_fastq_gz"         "r1_fastq_gz"        "r2_fastq_gz"        "sede"               "nn"                
[25] "r1_sample"          "r2_sample"          "sample_dir2"        "r1_fastq_gz_size"   "r2_fastq_gz_size"   "r1_sample_size_cal" "r2_sample_size_cal" "mergeR1_jobname"   
[33] "mergeR1_oo"         "mergeR1_eo"         "mergeR1_cmd"        "mergeR2_jobname"    "mergeR2_oo"         "mergeR2_eo"         "mergeR2_cmd"        "r1_trimmed"        
[41] "r2_trimmed"         "cutadapt_jobname"   "cutadapt_oo"        "cutadapt_eo"        "cutadapt_cmd"       "fname_pre"          "bow2_jobname"       "bow2_oo"           
[49] "bow2_eo"            "sam_file"           "sam_report_file"    "bow2_cmd"           "sam2bam_jobname"    "sam2bam_oo"         "sam2bam_eo"         "bam_file"          
[57] "sam2bam_cmd"        "bam2bed_jobname"    "bam2bed_oo"         "bam2bed_eo"         "bed_file"           "bam2bed_cmd"        "wig_file"           "fac"               
[65] "fac_n"              "fac_nn"             "bed2wig_jobname"    "bed2wig_oo"         "bed2wig_eo"         "bed2wig_cmd"        "bigwig_file"        "bed2wig2_jobname"  
[73] "bed2wig2_oo"        "bed2wig2_eo"        "bed2wig2_cmd"       "macs2_jobname"      "macs2_oo"           "macs2_eo"           "macs2_cmd"          "peak_file"

In [10]:
table(dsn$cell_type, dsn$antibody)

           
            ATAC CTCF ERG FLI1 GATA2 H3K27ac H3K27Ac H3K27AC H3K27me3 H3K4me3 HiC IgG LMO2 LYL1 Pol2 PU.1 PU1 RNAseq RUNX1 STAG2 TAL1
  CMP          0    3   3    3     3       1       0       2        1       1   2   1    3    3    2    0   3      0     3     2    3
  GMP          0    3   3    3     3       1       0       2        1       1   2   1    3    3    2    0   3      0     3     2    3
  HSC          0    3   3    3     3       1       0       2        1       1   2   1    3    3    2    0   3      0     3     2    3
  Kasumi1      1    0   1    1     1       0       1       0        0       0   0   0    1    1    0    1   0      0     1     0    1
  KG1          1    0   1    1     1       0       1       0        0       0   0   1    1    1    0    1   0      0     1     0    1
  ME1          1    0   1    1     1       0       1       0        0       0   0   1    1    1    0    1   0      0     1     0    1
  MEP          0    3   3    3     3       1      